## Extensions and problems

### Import the required libraries and load datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api import ols

%matplotlib inline
plt.style.use("seaborn-white")

**Load `Advertising`, `Credit`, and `Auto` datasets**

Datasets available on https://www.statlearning.com/resources-first-edition

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Advertising.csv"

advertising_df = pd.read_csv(
    data_url, header=0, index_col=0
)  # read Boston data as pandas data frame

In [ ]:
credit_url = "https://github.com/pykale/transparentML/raw/main/data/Credit.csv"

credit_df = pd.read_csv(credit_url)
credit_df["Student2"] = credit_df.Student.map({"No": 0, "Yes": 1})
credit_df.head(3)

In [ ]:
auto_url = "https://github.com/pykale/transparentML/raw/main/data/Auto.csv"

auto_df = pd.read_csv(auto_url, na_values="?").dropna()
auto_df.info()

### Qualitative predictors

In the previous sections, the predictor variables are *quantitative*. For example, in the `Credit` dataset, the response is `Balance` (average credit card debt for each individual) and there are six quantitative predictors: `Age` , `Cards` (number of credit cards), `Education` (years of education), `Income` (in thousands of dollars), `Limit` (credit limit), and `Rating` (credit rating). The following code displays the pairwise relationships between these variables (Figure 3.6 in the ISLR book).

In [ ]:
sns.pairplot(
    credit_df[["Balance", "Age", "Cards", "Education", "Income", "Limit", "Rating"]]
)

However, in practice, predictor variables are not always quantitative. For example in `Credit` dataset, it also contains four qualitative variables: `Student` (`Yes` or `No`), `Own` (`Yes` or `No`), `Married` (`Yes` or `No`), and `Region` (`South`, `East`, or `West`).

**Predictors with only two levels**

Run the following Least squares coefficient estimates associated with the regression of `Balance` onto `Own` in the `Credit` data set (Table 3.7 in the ISLR book).

In [ ]:
est = ols("Balance ~ Own", credit_df).fit()
est.summary().tables[1]

**Predictors with more than two levels**

Run the following Least squares coefficient estimates associated with the regression of `Balance` onto `Region` in the `Credit` data set (Table 3.8 in the ISLR book).

In [ ]:
est = ols("Balance ~ Region", credit_df).fit()
est.summary().tables[1]

### Extensions of linear regression

#### Interaction between variables

In the previous analysis of `Advertising` dataset, the predictor variables are assumed to be independent. However, this model may be incorrect. For example, `radio` advertising can increase the effectiveness of `TV` advertising, which is known as *interaction* effect in statistics. Consider the a standard multiple linear regression with two variables

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \epsilon.
$$

This model can be extended to include an interaction term between $X_1$ and $X_2$:

\begin{align}
Y &= \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_1 X_2 + \epsilon \\
&= \beta_0 + (\beta_1 + \beta_3 X_2) X_1 + \beta_2 X_2 + \epsilon \\
&= \beta_0 + \tilde{\beta}_1 X_1 + \beta_2 X_2 + \epsilon,
\end{align}

where $\tilde{\beta}_1 = \beta_1 + \beta_3 X_2$ is the *effective* coefficient on $X_1$. Then the association between $Y$ and $X_1$ is no longer constant and depends on the value of $X_2$. Similarly, the association between $Y$ and $X_2$ is no longer constant and depends on the value of $X_1$.

Run the following code for the `Advertising` data, least squares coefficient estimates associated with the regression of sales onto `TV` and `radio`, with an interaction term, `TV` $\times$ `radio` (Table 3.9 of the ISLR book).

In [ ]:
est = ols("Sales ~ TV + Radio + TV*Radio", advertising_df).fit()
est.summary().tables[1]

#### Interaction between qualitative and quantitative variables

The concept of interactions applies just as well to qualitative variables. In fact, sometimes an interaction between a qualitative variable and a quantitative variable has a particularly nice interpretation.

Run the following code to predict `balance` using the `income` (quantitative) and `student` (qualitative) variables, and compare the differences between including and excluding the interaction term between `income` and `student` (Figure 3.7 of the ISLR book).

In [ ]:
est1 = ols("Balance ~ Income + Student2", credit_df).fit()
est2 = ols("Balance ~ Income + Income*Student2", credit_df).fit()

print("Regression 1 - without interaction term")
print(est1.params)
print("\nRegression 2 - with interaction term")
print(est2.params)

In [ ]:
# Income (x-axis)
income = np.linspace(0, 150)

# Balance without interaction term (y-axis)
student1 = np.linspace(
    regr1["Intercept"] + regr1["Student2"],
    regr1["Intercept"] + regr1["Student2"] + 150 * regr1["Income"],
)
non_student1 = np.linspace(
    regr1["Intercept"], regr1["Intercept"] + 150 * regr1["Income"]
)

# Balance with iteraction term (y-axis)
student2 = np.linspace(
    regr2["Intercept"] + regr2["Student2"],
    regr2["Intercept"]
    + regr2["Student2"]
    + 150 * (regr2["Income"] + regr2["Income:Student2"]),
)
non_student2 = np.linspace(
    regr2["Intercept"], regr2["Intercept"] + 150 * regr2["Income"]
)

# Create plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(income, student1, "r", income, non_student1, "k")
ax2.plot(income, student2, "r", income, non_student2, "k")

for ax in fig.axes:
    ax.legend(["student", "non-student"], loc=2)
    ax.set_xlabel("Income")
    ax.set_ylabel("Balance")
    ax.set_ylim(ymax=1550)

#### Non-linear relationships

In some cases, the true relationship between the response and the predictors may be non-linear. Here we present a very simple way to directly extend the linear model to accommodate non-linear relationships, using *polynomial regression*.

Run the following code to display the relationship between `mpg` (gas mileage in miles per gallon) and different degrees of `horsepower` (Figure 3.8 of the ISLR book) for a number of cars in the `Auto` dataset.

In [ ]:
# With Seaborn's regplot() you can easily plot higher order polynomials.
plt.scatter(
    auto_df.horsepower, auto_df.mpg, facecolors="None", edgecolors="k", alpha=0.5
)
sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Linear",
    scatter=False,
    color="orange",
)
sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Degree 2",
    order=2,
    scatter=False,
    color="lightblue",
)
sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Degree 5",
    order=5,
    scatter=False,
    color="g",
)
plt.legend()
plt.ylim(5, 55)
plt.xlim(40, 240);

The figure seem to suggest that a quadratic relationship between `mpg` and `horsepower` might be more appropriate than a linear relationship. We can fit such a model using the following regression:

\begin{equation}
\textrm{mpg} = \beta_0 + \beta_1 \times \textrm{horsepower} + \beta_2 \times \textrm{horsepower}^2 + \epsilon.
\end{equation}

In fact, this is still a linear model with $X_1 = \textrm{horsepower}$ and $X_2 = \textrm{horsepower}^2$. Run the following code to fit a linear regression model with `mpg` as the response and `horsepower` and `horsepower^2` as the predictors (Table 3.10 of the ISLR book).

In [ ]:
auto_df["horsepower2"] = auto_df.horsepower**2
auto_df.head(3)

In [ ]:
est = ols("mpg ~ horsepower + horsepower2", auto_df).fit()
est.summary().tables[1]

### Problems of linear regression

- Non-linearity of the Data
  
  The linear regression model assumes that there is a straight-line relationship between the predictors and the response. If the true relationship is far from linear, then virtually all of the conclusions that we draw from the fit are suspect. In addition, the prediction accuracy of the model can be significantly reduced.

### Figure 3.9

In [ ]:
regr = LinearRegression()

# Linear fit
X = auto_df.horsepower.values.reshape(-1, 1)
y = auto_df.mpg
regr.fit(X, y)

auto_df["pred1"] = regr.predict(X)
auto_df["resid1"] = auto_df.mpg - auto_df.pred1

# Quadratic fit
X2 = auto_df[["horsepower", "horsepower2"]].values
regr.fit(X2, y)

auto_df["pred2"] = regr.predict(X2)
auto_df["resid2"] = auto_df.mpg - auto_df.pred2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
sns.regplot(
    x=auto_df.pred1,
    y=auto_df.resid1,
    lowess=True,
    ax=ax1,
    line_kws={"color": "r", "lw": 1},
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax1.hlines(
    0,
    xmin=ax1.xaxis.get_data_interval()[0],
    xmax=ax1.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax1.set_title("Residual Plot for Linear Fit")

# Right plot
sns.regplot(
    x=auto_df.pred2,
    y=auto_df.resid2,
    lowess=True,
    line_kws={"color": "r", "lw": 1},
    ax=ax2,
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax2.hlines(
    0,
    xmin=ax2.xaxis.get_data_interval()[0],
    xmax=ax2.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax2.set_title("Residual Plot for Quadratic Fit")

for ax in fig.axes:
    ax.set_xlabel("Fitted values")
    ax.set_ylabel("Residuals")

- Collinearity
  
  Collinearity refers to the situation in which two or more predictor variables are closely related to one another. 
  
  Run the following code to illustrate the problem of collinearity in the `Credit` dataset (Figure 3.14 of the ISLR book). In the left-hand panel, the two predictors `limit` and `age` appear to have no obvious relationship. In contrast, in the right-hand panel, the predictors `limit` and `rating` are very highly correlated with each other, and we say that they are *collinear*. In this context, `limit` and `rating` tend to increase or decrease together, it can be difficult to determine how each one separately is associated with the response, `balance`.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
ax1.scatter(credit_df.Limit, credit_df.Age, facecolor="None", edgecolor="r")
ax1.set_ylabel("Age")

# Right plot
ax2.scatter(credit_df.Limit, credit_df.Rating, facecolor="None", edgecolor="r")
ax2.set_ylabel("Rating")

for ax in fig.axes:
    ax.set_xlabel("Limit")
    ax.set_xticks([2000, 4000, 6000, 8000, 12000])

Run following code to learn the difficulties that can result from collinearity in the context of the `Credit` dataset (Table 3.11 of the ISLR book).

In [ ]:
y = credit_df.Balance

# Regression for left plot
X = credit_df[["Age", "Limit"]].values
regr1 = LinearRegression()
regr1.fit(scale(X.astype("float"), with_std=False), y)
print("Age/Limit\n", regr1.intercept_)
print(regr1.coef_)

# Regression for right plot
X2 = credit_df[["Rating", "Limit"]].values
regr2 = LinearRegression()
regr2.fit(scale(X2.astype("float"), with_std=False), y)
print("\nRating/Limit\n", regr2.intercept_)
print(regr2.coef_)

In [ ]:
# Create grid coordinates for plotting
B_Age = np.linspace(regr1.coef_[0] - 3, regr1.coef_[0] + 3, 100)
B_Limit = np.linspace(regr1.coef_[1] - 0.02, regr1.coef_[1] + 0.02, 100)

B_Rating = np.linspace(regr2.coef_[0] - 3, regr2.coef_[0] + 3, 100)
B_Limit2 = np.linspace(regr2.coef_[1] - 0.2, regr2.coef_[1] + 0.2, 100)

X1, Y1 = np.meshgrid(B_Limit, B_Age, indexing="xy")
X2, Y2 = np.meshgrid(B_Limit2, B_Rating, indexing="xy")
Z1 = np.zeros((B_Age.size, B_Limit.size))
Z2 = np.zeros((B_Rating.size, B_Limit2.size))

Limit_scaled = scale(credit_df.Limit.astype("float"), with_std=False)
Age_scaled = scale(credit_df.Age.astype("float"), with_std=False)
Rating_scaled = scale(credit_df.Rating.astype("float"), with_std=False)

# Calculate Z-values (RSS) based on grid of coefficients
for (i, j), v in np.ndenumerate(Z1):
    Z1[i, j] = (
        (y - (regr1.intercept_ + X1[i, j] * Limit_scaled + Y1[i, j] * Age_scaled)) ** 2
    ).sum() / 1000000

for (i, j), v in np.ndenumerate(Z2):
    Z2[i, j] = (
        (y - (regr2.intercept_ + X2[i, j] * Limit_scaled + Y2[i, j] * Rating_scaled))
        ** 2
    ).sum() / 1000000

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

min_RSS = r"$\beta_0$, $\beta_1$ for minimized RSS"

# Left plot
CS = ax1.contour(X1, Y1, Z1, cmap=plt.cm.Set1, levels=[21.25, 21.5, 21.8])
ax1.scatter(regr1.coef_[1], regr1.coef_[0], c="r", label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")
ax1.set_ylabel(r"$\beta_{Age}$", fontsize=17)

# Right plot
CS = ax2.contour(X2, Y2, Z2, cmap=plt.cm.Set1, levels=[21.5, 21.8])
ax2.scatter(regr2.coef_[1], regr2.coef_[0], c="r", label=min_RSS)
ax2.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")
ax2.set_ylabel(r"$\beta_{Rating}$", fontsize=17)
ax2.set_xticks([-0.1, 0, 0.1, 0.2])

for ax in fig.axes:
    ax.set_xlabel(r"$\beta_{Limit}$", fontsize=17)
    ax.legend()

Left: A contour plot of RSS for the regression of `balance` onto `age` and `limit`. The minimum value is well defined. Right: A contour plot of RSS for the regression of `balance` onto `rating` and `limit`. Because of the collinearity, there are many pairs ($\beta_{\text{Limit}}$ , $\beta_{\text{Rating}}$) with a similar value for RSS.

Collinearity reduces the accuracy of the estimates of the regression coefficients, it causes the standard error for $\hat{\beta}_j$ to grow. in the presence of collinearity, we may fail to reject $H_0$ : $\hat{\beta}_j = 0$. This means that the power of the hypothesis test—the probability of correctly power detecting a non-zero coefficient is reduced by collinearity.

Run the following code to compare the coefficient estimates obtained from two separate multiple regression models, where the first model is a regression of `balance` on `age` and `limit`, and the second model is a regression of `balance` on `rating` and `limit`.

In [ ]:
est_age_limit = ols("Balance ~ Age + Limit", credit_df).fit()
est_age_limit.summary().tables[1]

In [ ]:
est_rating_limit = ols("Balance ~ Rating + Limit", credit_df).fit()

# est_limit = ols("Limit ~ Age + Rating", credit_df).fit()

# print(1 / (1 - est_rating.rsquared))
# print(1 / (1 - est_age.rsquared))
# print(1 / (1 - est_limit.rsquared))

est_rating_limit.summary().tables[1]

```{admonition} How to interpret the results? 
:class: tip, dropdown
In the first regression, both `age` and `limit` are highly significant with very small p-values. In the second, the collinearity between `limit` and `rating` has caused the standard error for the `limit` coefficient estimate to increase by a factor of 12 and the p-value to increase to 0.701. In other words, the importance of the `limit` variable has been masked due to the presence of collinearity.
```

### Exercise

1. This question involves the use of simple linear regression on the **Auto** dataset from [Tabel 1.2](https://pykale.github.io/transparentML/01-intro/organisation.html#datasets-table).

    (a) Use the **LinearRegression()** function to perform a simple linear regression with **mpg** as the response and **horsepower** as the predictor. Use the **summary()** function to print the results. Comment on the output.
    
      For example:

      >i. Is there a relationship between the predictor and the response?  

      >ii. How strong is the relationship between the predictor and the response? 

      >iii. Is the relationship between the predictor and the response positive or negative? 

      >iv. What is the predicted __mpg__ associated with a **horsepower** of 98? What are the associated 95% confidence and prediction intervals?
        
    (b) Plot the response and predictor. Use the **regplot()** function to display the least-squared regression line.
    
    (c) Produce the diagnostic plots of the least squares regression fit. Comment on any problems you see with the fit. 

   
2. This question involves the use of multiple linear regression on the **Carseats** dataset from Tabel 1.2.

    (a) Fit a multiple regression model to predict **Sales** using **Price**, **Urban**, and **US**.
    
    (b) Provide an interpretation of each coefficient in the model. Be careful--some of the variables in the model are qualitative!       
    
    (c) Write out the model in equation form, being careful to handle the qualitative variables properly. 
    
    (d) For which of the predictors can you reject the null hypothesis $H_0 : \beta_j = 0$ ?  
    
    (e) On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.
    
    (f) How well do the models in (a) and (e) fit the data?
    
    (g) Using the model from (e), obtain 95% confidence intervals for the coefficient(s).
    
    (h) Is there evidence of outliers or high-leverage observations in the model from (e)?  

  
3. This question involves the **Boston** dataset from [Tabel 1.2](https://pykale.github.io/transparentML/01-intro/organisation.html#datasets-table). We will now try to predict per capita crime rate using the other variables in this dataset. In other owrds, per capita crime rate is the response, and the other variables are the prediction.

    (a) For each predictor, fit a simple linear regression model to predict the response. Describe your results. In which of the models is there a statistically significant association between the predictor and the response? Create some plots to back up your assertions.
    
    (b) Fit a multiple regression model to predict the response using all of the predictors. Describe your results.. FOr which predictiors can we reject the null hypothesis $H_0 : \beta_j = 0$ ? 
    
    (c) How do your results from (a) compare to your results form (b)? Create a plot displaying the univariate regression coefficients from (a) on the x-axis, and the multiple regression coefficients from (b) on the y-axis. That is, each predictor is displayed as a single point in the plot. Its coefficient in a simple linear regression model is shown on the x-axis, and its coefficient estimate in the multiple linear regression model is shown on the y-axis.
    
    (d) Is there evidence of non-linear association between any of the predictors and the response? To answer this question, for each predictor X, fit a model of the form, 
    
    <center> $$Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \epsilon$$
    
    